# Deep Learning segmentation suite dessigned for 2D microscopy image segmentation
---

<font size = 4>The corresponging **[GitHub repository](https://github.com/esgomezm/microscopy-dl-suite-tf)**, developed by the authors of the paper. 

---
<font size = 4>**Please cite the corresponding paper** 

## Install required libraries
<font size = 4>In case it returns "restart session" at the end fo the installation, please click it and start from the beginning.

In [1]:
!git clone https://github.com/esgomezm/microscopy-dl-suite-tf
!pip3 install -r microscopy-dl-suite-tf/dl-suite/requirements.txt

Cloning into 'microscopy-dl-suite-tf'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 220 (delta 102), reused 60 (delta 29), pack-reused 0
Receiving objects: 100% (220/220), 4.27 MiB | 4.01 MiB/s, done.
Resolving deltas: 100% (102/102), done.
     |████████████████████████████████| 47.6 MB 74 kB/s 
     |████████████████████████████████| 516.2 MB 4.4 kB/s 
     |████████████████████████████████| 48.4 MB 1.5 MB/s 
     |████████████████████████████████| 101 kB 10.7 MB/s 
     |████████████████████████████████| 230 kB 47.9 MB/s 
     |████████████████████████████████| 454 kB 55.7 MB/s 
     |████████████████████████████████| 3.0 MB 11.5 MB/s 
     |████████████████████████████████| 2.9 MB 50.9 MB/s 
     |████████████████████████████████| 40 kB 6.0 MB/s 
     |████████████████████████████████| 1.0 MB 53.7 MB/s 
     |████████████████████████████████| 802 kB 55.9 MB/s 
     |███████

## Alternative: Mount your drive and work with the information there

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
path_code = "Path where the code is stored"
os.chdir(path_code)
!pip3 install -r microscopy-dl-suite-tf/dl-suite/requirements.txt

# Download example data

It will download the example data from a [Zenodo repository](https://zenodo.org/record/5777994) and unzip it.

If you use this data, please cite the corresponding publication

In [ ]:
!wget https://zenodo.org/record/5777994/files/HT1080WT_PHASECONTRAST.zip?download=1

--2021-12-14 20:03:22--  https://zenodo.org/record/5777994/files/HT1080WT_PHASECONTRAST.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3557453821 (3.3G) [application/octet-stream]
Saving to: ‘HT1080WT_PHASECONTRAST.zip?download=1’

RAST.zip?download=1  41%[=======>            ]   1.37G  3.71MB/s    eta 5m 14s 

In [ ]:
!unzip HT1080WT_PHASECONTRAST.zip

# Train

In [ ]:
import json
import time
import os

json_template ='/content/drive/MyDrive/Projectos/3D-PROTUCEL/Code/MU_Lux_CZ/microscopy-dl-suite-tf/examples/config/config_template.json'

training_data = "/content/drive/MyDrive/BiiG/3D-PROTUCEL/data/test"
validation_data = "/content/drive/MyDrive/BiiG/3D-PROTUCEL/data/val"
test_data = "/content/drive/MyDrive/BiiG/3D-PROTUCEL/data/test"

## OPTIONS FOR MODEL ARCHITECTURES. TRAINING DATA NEEDS TO CORRESPOND TO THE 
# INPUT-OUTPUT FORMAT (2D IMAGES OR 2D VIDEO FILES)
# ---------------------------------------------------------

# - 'mobilenet_mobileunet_lstm': A pretrained mobilenet in the encoder with skip connections to the decoder of a mobileunet and a ConvLSTM layer at the end that will make the entire architecture recursive.
# - 'mobilenet_mobileunet': A pretrained mobilenet in the encoder with skip connections to the decoder of a mobileunet (2D).
# - 'unet_lstm': 2D U-Net with ConvLSTM units in the contracting path.
# - 'categorical_unet_transpose': 2D U-Net for different labels ({0}, {1}, ...) with transpose convolutions instead of upsampling.
# - 'categorical_unet_fc_dil': 2D U-Net for different labels ({0}, {1}, ...) with fully connected dilated convolutions.
# - 'categorical_unet_fc': 2D U-Net for different labels ({0}, {1}, ...) with fully connected convolutions.
# - 'categorical_unet': 2D U-Net for different labels ({0}, {1}, ...).
# - 'unet' or "None": 2D U-Net with a single output.
model_architecture = "mobilenet_mobileunet_lstm"

## TRAINING PARAMETERS
training_epochs = 200
learning_rate = 0.0005
# 1 if we want to freeze the mobilenet encoder to 
# apply some transfer learning, 0 if we want to train the entire network:
freeze_pretrained_encoder = 1 
## TOTAL BATCH SIZE: patch_batch*bach_size
patch_batch = 1 # Number of patches cropped from each image on each epoch.
bach_size = 2 # Number of images to take on each epoch
spatial_input_shape = [256, 256]


# CREATE THE CONFIGURATION FILE
with open(json_template, "r") as jsonFile:
    data = json.load(jsonFile)
data["cnn_name"] = model_architecture
# data["TRAINPATH"] = training_data
# data["TESTPATH"] = test_data
# data["VALPATH"] = validation_data
data["model_train_decoder_only"] = freeze_pretrained_encoder
data["model_lr"] = learning_rate
data["train_max_epochs"] = training_epochs
data["datagen_patch_batch"] = patch_batch
data["datagen_batch_size"] = bach_size
data["datagen_dim_size"] = spatial_input_shape

# !python MU_Lux_CZ/microscopy-dl-suite-tf/dl-suite/train.py '/content/my_config.json'
for f in range(20):
    main_path = "/content/drive/MyDrive/Projectos/3D-PROTUCEL/Code/MU_Lux_CZ/training_replicates/fix_model_seed/"
    output_path = os.path.join(main_path, "replica_{}.json".format(f))
    
    data["OUTPUTPATH"] = output_path
    data["model_seed_initializer"] = 10
    data["train_seed_initializer"] = None
    
    if not os.path.exists(output_path):
      os.mkdir(output_path)
    new_json = os.path.join(output_path, "my_config.json")

    with open('/content/my_config.json', "w") as jsonFile:
        json.dump(data, jsonFile)

    with open(new_json, "w") as jsonFile:
        json.dump(data, jsonFile)   
    !python /content/drive/MyDrive/Projectos/3D-PROTUCEL/Code/MU_Lux_CZ/microscopy-dl-suite-tf/dl-suite/train.py '/content/my_config.json'

# !python MU_Lux_CZ/microscopy-dl-suite-tf/dl-suite/train.py '/content/my_config.json'
for f in range(20):
    main_path = "/content/drive/MyDrive/Projectos/3D-PROTUCEL/Code/MU_Lux_CZ/training_replicates/fix_training_seed/"
    output_path = os.path.join(main_path, "replica_{}.json".format(f))
    
    data["OUTPUTPATH"] = output_path
    data["model_seed_initializer"] = None
    data["train_seed_initializer"] = 10
    
    if not os.path.exists(output_path):
      os.mkdir(output_path)
    new_json = os.path.join(output_path, "my_config.json")

    with open('/content/my_config.json', "w") as jsonFile:
        json.dump(data, jsonFile)

    with open(new_json, "w") as jsonFile:
        json.dump(data, jsonFile)   
    !python /content/drive/MyDrive/Projectos/3D-PROTUCEL/Code/MU_Lux_CZ/microscopy-dl-suite-tf/dl-suite/train.py '/content/my_config.json'


Seed 10 fixed for model initialization
{'n_filters': 16, 'pools': 3, 'kernel_size': [3, 3], 'dilation_rate': 2, 'mobile_alpha': 0.35, 'time_windows': 5, 'lr': 0.0005, 'dropout': 0.01, 'activation': 'elu', 'last_activation': 'tanh', 'padding': 'same', 'kernel_initializer': 'glorot_uniform', 'lossfunction': 'sparse_cce', 'loss_tips': 'L1L2', 'metrics': 'accuracy', 'train_decoder_only': 1, 'category_weights': [1, 10], 'seed_initializer': 10}
2021-11-07 22:51:05.412059: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-07 22:51:05.434455: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-07 22:51:05.435109: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreC

In [ ]:
# import os
# os.chdir("/content/drive/My Drive/Projectos/3D-PROTUCEL/Code/MU_Lux_CZ/microscopy-dl-suite-tf/dl-suite")
# import sys
# from internals.callbacks import initiate_callbacks, ImagesTensorboardCallback
# from utils.read_config import Dict2Obj
# from internals.build_processed_videos import build_videos_CTC
# from internals.tiling_strategy import model_prediction, model_prediction_lstm
# from models.builder import build_model
# from data_generators.build_data import generate_data
# from internals.build_processed_videos import build_videos
# from numpy import random
# from tensorflow.random import set_seed

# # Read the configuration file with all the metadata and information about the training.
# PATH2CONFIG = '/content/my_config.json'
# # PATH2CONFIG = 'trained_config/config_docker_local.json'

# config = Dict2Obj(PATH2CONFIG)

# # Fix a seed if we want a model to be initialize always in the same manner.
# # Tensorflow has its own random generator that needs also to be seeded.
# if config.model_seed_initializer != "None":
#     print("Seed {} fixed for model initialization".format(config.model_seed_initializer))
#     random.seed( config.model_seed_initializer )
#     set_seed( config.model_seed_initializer )

# ## BUILD KERAS MODEL
# # ----------------------------------------------
# keras_model = build_model(config)


# ## DEFINE DATA GENERATION
# # ----------------------------------------------
# training_generator, validation_generator = generate_data(config)

# # Create some shots for Tensorboard
# B = validation_generator.batch_size
# validation_generator.batch_size = 10
# val_x, val_y = validation_generator.__getitem__(0)
# validation_generator.batch_size = B

# # Define callbacks and load pretrained weights
# Itb = ImagesTensorboardCallback(val_x, val_y, os.path.join(config.OUTPUTPATH, 'logs/tmp/'), n_images=10, step=20)
# # del val_x, val_y

# if config.train_pretrained_weights != "None":
#     last_epoch = config.train_pretrained_weights
#     if last_epoch.__contains__('/'):
#         last_epoch = last_epoch.split('/')[-1]
#     last_epoch = last_epoch.split('.')[0]
#     callbacks = initiate_callbacks(config, keras_model, last_epoch=last_epoch)
# else:
#     callbacks = initiate_callbacks(config, keras_model)
# callbacks.append(Itb)

# # SET A SEED FOR THE DATA ENTERING THE TRAINING
# # ----------------------------------------------
# if config.train_seed_initializer != "None":
#     print("Seed {} fixed for training data generator".format(config.train_seed_initializer))
#     # Fix a seed if we want data generator to be initilized always in the same way.
#     random.seed( config.train_seed_initializer )
#     set_seed( config.train_seed_initializer )
# else:
#     # We need a random value just in case we fixed the model initializer.
#     # Otherwise, we would be fixing always the data generator in the same way.
#     random.seed(random.randint(100000))
#     set_seed(random.randint(100000))

# # Train the model
# # ----------------------------------------------
# training_generator.batch_size = 10
# # x, y = training_generator.__getitem__(0)

# keras_model.fit(training_generator, validation_data=(val_x, val_y), 
#                 epochs=2,
#                 validation_batch_size=config.datagen_batch_size,#validation_steps=config.datagen_batch_size,
#                 callbacks=callbacks)

# Run the training using a .json file that contains all the details

In [ ]:
# !python microscopy-dl-suite-tf/dl-suite/train.py 'microscopy-dl-suite-tf/examples/config/config_mobilenet_lstm_5.json' 